## Getting ready

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

2022-01-24 14:40:52.528219: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## How to do it...

In [2]:
breast_cancer = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'
path = tf.keras.utils.get_file(breast_cancer.split("/")[-1], breast_cancer)

columns = ['sample_code', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity',
          'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei', 'bland_chromatin',
          'normal_nucleoli', 'mitoses', 'class']

data = pd.read_csv(path, header=None, names=columns, na_values=[np.nan, '?'])
data = data.fillna(data.median())

np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = (train['class']==4).astype(int)
train.drop(['sample_code', 'class'], axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = (test['class']==4).astype(int)
test.drop(['sample_code', 'class'], axis=1, inplace=True)

24576/19889 [=====================================] - 0s 3us/step


In [3]:
data

,sample_code,clump_thickness,cell_size_uniformity,cell_shape_uniformity,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2.0,1,1,1,2
695,841769,2,1,1,1,2,1.0,1,1,1,2
696,888820,5,10,10,3,7,3.0,8,10,2,4
697,897471,4,8,6,4,3,4.0,10,6,1,4


In [18]:
def create_logreg(feature_columns, feature_layer_inputs, optimizer, loss='binary_crossentropy',
                  metrics=['accuracy'], l2=0.01):
    
    regularizer = keras.regularizers.l2(l2)
    
    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1,
                                kernel_initializer='normal',
                                kernel_regularizer = regularizer,
                                activation='sigmoid')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs = outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model

In [19]:
def define_feature_columns_layers(data_df, categorical_cols, numeic_cols):
    feature_columns = []
    feature_layer_inputs = {}
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name)
        
    for feature_name in categorical_cols:
        vocabulary = data_df[feature_name].unique()
        cat = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        cat_one_hot = tf.feature_column.indicator_column(cat)
        feature_columns.append(cat_one_hot)
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.int32)
        
    return feature_columns, feature_layer_inputs

In [20]:
import tempfile

def canned_keras(model):
    model_dir = tempfile.mkdtemp()
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir=model_dir)
    return keras_estimator

In [21]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=256):
    
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
        if shuffle:
            ds = ds.shuffle(1000)
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    
    return input_function

In [25]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity',
               'cell_shape_uniformity',
               'marginal_adhesion', 'single_epithelial_cell_size',
               'bare_nuclei', 'bland_chromatin',
               'normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_columns_layers(data, categorical_cols, numeric_cols)

optimizer = keras.optimizers.Ftrl(learning_rate=0.007)
model = create_logreg(feature_columns, feature_layer_inputs, optimizer, l2=0.01)

estimator = canned_keras(model)

train_input_fn = make_input_fn(train, y_train, num_epochs=300, batch_size=32)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)
estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)

print(result)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpe05asa7g', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Va

2022-01-24 15:01:23.670294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:23.670531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-24 15:01:23.670580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:23.670796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:23.670977: I tensorflow/core/common

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='/tmp/tmpe05asa7g/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: /tmp/tmpe05asa7g/keras/keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmpe05asa7g/model.ckpt.


2022-01-24 15:01:24.057305: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:24.057552: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-24 15:01:24.057618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:24.057838: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:24.058024: I tensorflow/core/common

INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...


2022-01-24 15:01:24.344036: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11


INFO:tensorflow:loss = 0.752352, step = 0
INFO:tensorflow:global_step/sec: 650.75
INFO:tensorflow:loss = 0.6747127, step = 100 (0.154 sec)


2022-01-24 15:01:24.622801: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


INFO:tensorflow:global_step/sec: 898.919
INFO:tensorflow:loss = 0.67483294, step = 200 (0.111 sec)
INFO:tensorflow:global_step/sec: 934.372
INFO:tensorflow:loss = 0.6841604, step = 300 (0.107 sec)
INFO:tensorflow:global_step/sec: 917.207
INFO:tensorflow:loss = 0.67026734, step = 400 (0.109 sec)
INFO:tensorflow:global_step/sec: 899.612
INFO:tensorflow:loss = 0.5993425, step = 500 (0.111 sec)
INFO:tensorflow:global_step/sec: 913.465
INFO:tensorflow:loss = 0.6195815, step = 600 (0.109 sec)
INFO:tensorflow:global_step/sec: 942.994
INFO:tensorflow:loss = 0.536618, step = 700 (0.106 sec)
INFO:tensorflow:global_step/sec: 933.634
INFO:tensorflow:loss = 0.4947418, step = 800 (0.107 sec)
INFO:tensorflow:global_step/sec: 924.318
INFO:tensorflow:loss = 0.44216847, step = 900 (0.108 sec)
INFO:tensorflow:global_step/sec: 906.694
INFO:tensorflow:loss = 0.43833935, step = 1000 (0.110 sec)
INFO:tensorflow:global_step/sec: 941.313
INFO:tensorflow:loss = 0.34753132, step = 1100 (0.106 sec)
INFO:tensorflo

/home/wil/tensorflow/venv/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2426: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
2022-01-24 15:01:30.680883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-24 15:01:30.681121: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.29GiB/s
2022-01-24 15:01:30.681167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1